In [62]:
import paramiko
from getpass import getpass

#Create the password prompt
thePass = getpass(prompt = "Please enter your SSH password: ")

#Host information
host = "192.168.6.71"
port = 2222
username = "jahseem.maxwell"
password = thePass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authenticated failed.")

In [39]:
sftp = ssh.open_sftp()

sftp.put('/Users/jahseem/SYS-320-SP22/Week7/homework/kraken', '/home/jahseem.maxwell/kraken')



<SFTPAttributes: [ size=9962192 uid=1286604136 gid=1286603265 mode=0o100644 atime=1646852061 mtime=1646852152 ]>

In [53]:
#Task 2

kcommand = "chmod +x kraken && sudo -S /home/jahseem.maxwell/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin"

stdin, stdout, stderr = ssh.exec_command(kcommand)

stdin.write(thePass + "\n")

lines = stdout.readlines()

output = ''.join(lines)

print(output)

with open('kraken_results.txt', 'a') as f:
    f.write(output)

# The commands 


INFO[0000] Looking for configuration file with name config.yaml 
INFO[0000] No configuration file found, generating a default one... 
INFO[0000] Loading Yara rules...                        
INFO[0000] Scanning running processes...                
WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=3138 process=ls
WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=25680 process=snapd
INFO[0014] Scanning autoruns...                         
WARN[0015] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
WARN[0015] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
INFO[0015] Scanning the filesystem (this can take several minutes)... 
ERRO[0015] Some malicious artifacts have been detected on this system: 
ERRO[0015] Found detection for foundGoBinary            
ERRO[0015] Found detection for foundGoBinary            
ERRO[0015] Found detection for foundGoBinary      

In [59]:
#Task 3
lsofCMD = ["sudo -S lsof -p 3138", "sudo -S lsof -p 25680", "sudo -S lsof -p 3025", "sudo -S lsof -p 3027"]
#pidof *process* will give you the process ID
#ps aux will give you a list of running processes
#3025, 3027, and 3138 were all associated with the p.dalton user

def commander(commandField):

    # Run a list of commands
    for eachCMD in commandField:
        # Get the output from the command
        stdin, stdout, stderr = ssh.exec_command(eachCMD)

        stdin.write(thePass + "\n")

        # Get results from stdout
        lines = stdout.readlines()
    
        #Conver the list to a string
        output = ''.join(lines)

        #Header output
        sepHeader = '' + '### BEGIN ' + eachCMD + ' ###\n\n'

        # Footer
        sepFooter =  '' + '### END ' + eachCMD + ' ###\n\n'

        # Concatenate the header, output, and footer

        cmd_output = sepHeader + output + sepFooter

        # Save command output to a file
        with open('lsof_results.txt', 'a') as f:
            f.write(cmd_output)
            
commander(lsofCMD)

In [73]:
#Task 4
# Start SFTP session
sftp = ssh.open_sftp()

# List of files to download
files = ['/usr/lib/systemd/systemd',
 '/usr/lib/systemd/libsystemd-shared-245.so', 
 '/usr/local/bin/ls',
 '/home/p.dalton/test-lk/.d.log',
 '/home/p.dalton/test-lk/d.pid',
 '/usr/lib/snapd/snapd',
 '/usr/lib/x86_64-linux-gnu/libunistring.so.2.1.0',
 '/usr/lib/x86_64-linux-gnu/libcrypto.so.1.1',
 '/usr/lib/x86_64-linux-gnu/libgcrypt.so.20.2.5'
  ]

# Loop through the files and download time
for eachFile in files:

    # Create a list from the filename
    x = eachFile.split("/")

    # -1 is used to pluck the last element in a list
    filename = x[-1]

    print(filename)
    # Local path to store files
    local_path = "suspect_files/" + filename

    sftp.get(eachFile, local_path)
    


systemd
libsystemd-shared-245.so
ls
.d.log
d.pid
snapd
libunistring.so.2.1.0
libcrypto.so.1.1
libgcrypt.so.20.2.5
